In [1]:
import pandas as pd

In [4]:
impressions_df = pd.read_csv("impressions.сsv")
impressions_df.head()

,rank,event_datetime_m,is_clicked,id_show,id_user
0,1,2017-01-12 12:30:00,0.0,5547,906335
1,3,2017-01-12 14:53:00,0.0,5547,5038628
2,4,2017-01-12 17:10:00,0.0,8148,5038628
3,2,2017-01-12 14:48:00,0.0,8131,5038628
4,1,2017-01-12 14:58:00,0.0,8367,5038628


Первый DataFrame. По аналогичному надо и делать предсказание.Что можно сказать. Придётся переделать DateTime features. Show_id и user_id в данном случае бесполезные. Если представть их как категориальные признаки, то получится супер-разреженная матрица, в которой находится около 0 полезной инфы. Довольно логично будет их выкинуть в дальнейшем обучении, а вместо них вставить инфу из других df, в которых и содержатся паттерны, уточняющие ползователей и мероприятия. Что касается признака rank, он вроде категорильаный, но с другой стороны является показателем, какое место в списке занимает билет, поэтому его можно оставить как есть(StandardScaler)

In [25]:
show_df = pd.read_csv("show_data.сsv", index_col = 'id_show')
show_df.head()

,organizer_id,age_category,duration,parent_genre_id,child_genre_id,IdBuilding,show_maxprice,show_minprice,show_meanprice,show_stdprice
id_show,,,,,,,,,,
0,1854307.0,4.0,10200.0,[13654],[13655],10.0,6000.0,160.0,1495.231712,777.042344
1,1854309.0,2.0,10800.0,[13654],[13658 13659],12.0,2600.0,150.0,995.630174,676.837725
2,1854327.0,5.0,8700.0,[13654],[13655],33.0,10000.0,50.0,743.746006,1153.720468
3,1854327.0,2.0,6600.0,[13679],[1868812],33.0,3000.0,50.0,707.560152,605.466747
4,1854319.0,7.0,6600.0,[13654],[13657],22.0,450.0,300.0,384.545227,62.711090


Вот тут мы и добрались до шоу! Оно начинается. Начнём по порядку:
<ol>
    <li>organazier_id - 1200 категорий, но с другой стороны в среднем на одного организатора приходится 25 мероприятий - не так мало. Eсть два пути - OneHot. А можно забить на этот параметр и вместо него вставить пару простых категориальных - сделать групбай, посмотреть насколько организатор популярный, дать какой-то рейтинг.</li>
    <li>age_category - в названии есть слово category... Но на самом деле он похож на чиловой. надо попробовать и так и так.</li>
    <li>duration - в половине мероприятий не указана их длительность. Наверное стоит такие классифицировать отдельно. Средняя длительность - 4000 чего-то, на самом деле средняя длительность чуть больше 8000 чего-то. Если представить, что это секунды, то 8000/60=133 минуты, чуть больше двух часов. Логистическая регрессия вряд ли будет довольна этим признаком в таком виде, как он есть. Его легко можно сделать категориальным. Ещё раз поделить по часам и взять целую часть. Супер-долгие мероприятия лучше засунуть в один класс. Надо будет посмотреть распределение на всякие bar plotах</li>
    <li>parent_genre_id and child_genre_id - OneHot так и напрашивается. Только есть один момент. Делать его в лоб нельзя. Надо посчитать сколько всего на самом деле жанров, из-за того что по дефолту двойной жанр является как бы уникальнымю Предполагаю, что их порядка 50.</li>
    <li>IdBuilding - 602 категории... Предложение сделать тоже, что и с organaizer_id</li>
    <li>price features - обычные числовые признаки, полезно посмотреть на графики. может быть как-то удачно разобьётся на классы. С последним признаком </li>
</ol>

In [26]:
print(show_df.shape)
print(show_df['organizer_id'].nunique())
print(show_df['age_category'].value_counts())
print(show_df['duration'].describe())
print(len(show_df['duration'].nonzero()[0]))
print(show_df['parent_genre_id'].nunique())
print(show_df['child_genre_id'].nunique())
print(show_df['IdBuilding'].nunique())
print(show_df['show_maxprice'].describe())
print(show_df['show_minprice'].describe())
print(show_df['show_meanprice'].describe())
print(show_df['show_stdprice'].describe())

(30949, 10)
1198
2.0     7976
3.0     5952
1.0     5515
4.0     3611
5.0     2805
6.0      366
7.0      242
8.0      164
9.0       63
13.0      33
14.0      13
15.0      10
12.0      10
11.0       2
Name: age_category, dtype: int64
count     30025.000000
mean       3999.137386
std        6698.175723
min           0.000000
25%           0.000000
50%           0.000000
75%        7200.000000
max      216000.000000
Name: duration, dtype: float64
15499
102
901
602
count     29931.000000
mean       2552.690856
std        4652.360170
min           1.000000
25%         800.000000
50%        1500.000000
75%        3000.000000
max      200000.000000
Name: show_maxprice, dtype: float64
count    29931.00000
mean       688.37817
std        797.18843
min          0.01000
25%        300.00000
50%        500.00000
75%        800.00000
max      30000.00000
Name: show_minprice, dtype: float64
count    29931.000000
mean      1274.532005
std       1341.747581
min          1.000000
25%        529.411765
5